In [1]:
#import package
import bs4
import requests
import pandas as pd
import geopandas as gpd
import random
import math
from geopy.distance import geodesic
import sqlalchemy as db
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import matplotlib

/Users/lixinrui/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Part1 Data Processing

## distance calculation
Define a function to caculate distcance between two pairs of given longtitude and latitude

In [2]:
#define a function to calculate distance
def create_distance(lat1,lon1,lat2,lon2):
    dist = geodesic((lat1,lon1),(lat2,lon2)).km
    return dist

## uber_date
1. Import and go through the data, delete the data out of given range
2. Crate a 'year' column to obatin the # of datapoints in ervery year
3. Drop invalid variables and add distance to the dataframe

In [3]:
#import uber_data
uber = pd.read_csv('uber_rides_sample.csv')

In [4]:
#check
uber.head(2)

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1


In [5]:
# keep data in required range
uber_select = uber[(uber['pickup_longitude'] >= -74.242330) & (uber['pickup_longitude'] <= -73.717047) \
    & (uber['pickup_latitude'] >= 40.560445) & (uber['pickup_latitude'] <= 40.908524) &\
    (uber['dropoff_longitude'] >= -74.242330) & (uber['dropoff_longitude'] <= -73.717047) \
    & (uber['dropoff_latitude'] >= 40.560445) & (uber['dropoff_latitude'] <= 40.908524)]

In [6]:
#will sample certain number of data from yellow taxi data based on this
uber_select['year'] = pd.to_datetime(uber['pickup_datetime']).dt.year
uber_select['year'].value_counts()

<ipython-input-6-77b182171db7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_select['year'] = pd.to_datetime(uber['pickup_datetime']).dt.year


2012    31531
2011    31102
2013    30496
2009    30077
2010    29494
2014    29258
2015    13514
Name: year, dtype: int64

In [7]:
#drop invalid variables and add distance to the dataframe
uber_select.drop(['Unnamed: 0','key','fare_amount','passenger_count', 'year'], axis = 1, inplace = True)

<ipython-input-7-4a960f229f9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_select.drop(['Unnamed: 0','key','fare_amount','passenger_count', 'year'], axis = 1, inplace = True)


In [8]:
uber_select['distance'] = uber_select.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)

<ipython-input-8-355f9ddcbd83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_select['distance'] = uber_select.apply(lambda x: create_distance(x['pickup_latitude'],\


In [9]:
uber_data = uber_select.reset_index()
uber_data = uber_data.drop(['index'], axis = 1)

In [10]:
uber_data['pickup_datetime'] = pd.to_datetime(uber_data['pickup_datetime'])

In [11]:
uber_data.head(2)

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1.681111
1,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,2.454363


## yellow_taxi data
1. Improt taxi_zone.shp and convert geometry data into normal longtitue and lantitude
2. Get the url of yellow taxi and collect the data as yearly file and check the data
3. Drop invalid variables
4. Drop the data with same PULocationID and DOLocationID, merge geographical information and drop data out of given range
5. Sample the data accoring to the data amouont in uber
6. Add distance to the dataframe
7. Because data in 2010 and 2009 have different information in it, we process them seperately

In [12]:
#import taxi_zone information
taxi_zone = gpd.read_file('taxi_zones.shp')

In [13]:
#check
taxi_zone.head(2)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."


In [14]:
#change geometry information to lantitude and longtitde
taxi_zone = taxi_zone.to_crs(4326)
taxi_zone['lon'] = taxi_zone.centroid.x  
taxi_zone['lat'] = taxi_zone.centroid.y

<ipython-input-14-770757aa3575>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone['lon'] = taxi_zone.centroid.x
<ipython-input-14-770757aa3575>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone['lat'] = taxi_zone.centroid.y


In [15]:
taxi_zone.head(2)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,lon,lat
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",-74.174000,40.691831
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745


In [16]:
#get the url of yellow taxi
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"


def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html


def find_taxi_parquet_links():
    html = get_taxi_html()
    soup = bs4.BeautifulSoup(html, 'html.parser')
    yellow_lst = [a['href'] for a in soup.find_all(title = 'Yellow Taxi Trip Records')]
    return yellow_lst

yellow_lst =  find_taxi_parquet_links()


In [17]:
#create the year_num list according to uber_data 
year_num = [13514, 29258, 30496, 31531, 31102, 29494, 30077]

In [18]:
#define the function to get and clean data in 2011-2015
def get_and_clean_data(n):
    file = list(range(1,13))
    for i in range(len(file)):
        file[i] = pd.read_parquet(yellow_lst[i+12*n-84])
    for i in range(len(file)):
        file[i].drop(['VendorID','tpep_dropoff_datetime','passenger_count',\
                            'trip_distance','RatecodeID','store_and_fwd_flag',\
                            'payment_type','fare_amount','extra','mta_tax',\
                            'tolls_amount','improvement_surcharge','total_amount',
                            'congestion_surcharge','airport_fee'], \
                           axis = 1, inplace = True)
    for i in range(len(file)):
        file[i].drop(file[i][file[i]['PULocationID'] == file[i]['DOLocationID']].index, inplace = True)
    for i in range(len(file)):
        file[i].rename(columns = {'PULocationID' : 'LocationID','tpep_pickup_datetime':\
                                 'pickup_datetime'}, inplace = True)
        file[i] = pd.merge(file[i], taxi_zone[['LocationID','lon','lat']], how = 'left', on = 'LocationID')
        file[i].drop(['LocationID'], axis = 1, inplace = True)
        file[i].rename(columns = {'DOLocationID' : 'LocationID','lon':'pickup_longitude', 'lat':\
                                       'pickup_latitude'}, inplace = True)
        file[i] = pd.merge(file[i], taxi_zone[['LocationID','lon','lat']], how = 'left', on = 'LocationID')
        file[i].drop(['LocationID'], axis = 1, inplace = True)
        file[i].rename(columns = {'lon':'dropoff_longitude', 'lat':'dropoff_latitude'}, inplace = True)
    for i in range(len(file)):
        file[i] = file[i][(file[i]['pickup_longitude'] >= -74.242330) \
                                & (file[i]['pickup_longitude'] <= -73.717047) \
                                & (file[i]['pickup_latitude'] >= 40.560445) \
                                & (file[i]['pickup_latitude'] <= 40.908524) \
                                &(file[i]['dropoff_longitude'] >= -74.242330) \
                                & (file[i]['dropoff_longitude'] <= -73.717047) \
                                & (file[i]['dropoff_latitude'] >= 40.560445) \
                                & (file[i]['dropoff_latitude'] <= 40.908524)]
    file_res = pd.DataFrame()
    for i in range(len(file)):
        file_res = pd.concat([file_res,file[i].sample(round(year_num[n]/12+1))], axis = 0)
    file_res = file_res.sample(year_num[n])
    file_res['distance'] = file_res.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)
    
    return file_res

In [19]:
#define the function to get and clean data in 2010
def get_and_clean_data2(n):
    file = list(range(1,13))
    for i in range(len(file)):
        file[i] = pd.read_parquet(yellow_lst[i+12*n-24])
    for i in range(len(file)):
        file[i].drop(['vendor_id','dropoff_datetime','passenger_count',\
                            'trip_distance','rate_code','store_and_fwd_flag',\
                            'payment_type','fare_amount','surcharge','mta_tax',\
                            'tolls_amount','total_amount'], axis = 1, inplace = True)
    for i in range(len(file)):
        file[i]['pickup_datetime'] = pd.to_datetime(file[i]['pickup_datetime'])
    for i in range(len(file)):
        file[i] = file[i][(file[i]['pickup_longitude'] >= -74.242330) \
                                & (file[i]['pickup_longitude'] <= -73.717047) \
                                & (file[i]['pickup_latitude'] >= 40.560445) \
                                & (file[i]['pickup_latitude'] <= 40.908524) \
                                &(file[i]['dropoff_longitude'] >= -74.242330) \
                                & (file[i]['dropoff_longitude'] <= -73.717047) \
                                & (file[i]['dropoff_latitude'] >= 40.560445) \
                                & (file[i]['dropoff_latitude'] <= 40.908524)]
    file_res = pd.DataFrame()
    for i in range(len(file)):
        file_res = pd.concat([file_res,file[i].sample(round(year_num[n+5]/12+1))], axis = 0)
    file_res = file_res.sample(year_num[n+5])
    file_res['distance'] = file_res.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)
    
    return file_res

In [20]:
#define the function to get and clean data in 2009
def get_and_clean_data3(n):
    file = list(range(1,13))
    for i in range(len(file)):
        file[i] = pd.read_parquet(yellow_lst[i+12*n-12])
    for i in range(len(file)):
        file[i].drop(['vendor_name','Trip_Dropoff_DateTime','Passenger_Count',\
                            'Trip_Distance','Rate_Code','store_and_forward',\
                            'Payment_Type','Fare_Amt','surcharge','mta_tax',\
                            'Tolls_Amt','Total_Amt'], axis = 1, inplace = True)
    for i in range(len(file)):
        file[i].rename(columns = {'Trip_Pickup_DateTime' : 'pickup_datetime'\
                       ,'Start_Lon' : 'pickup_longitude', 'Start_Lat' : 'pickup_latitude',\
                       'End_Lon' : 'dropoff_longitude','End_Lat' : 'dropoff_latitude','Tip_Amt':'tip_amount'},\
                       inplace = True)
    for i in range(len(file)):
        file[i]['pickup_datetime'] = pd.to_datetime(file[i]['pickup_datetime'])
    for i in range(len(file)):
        file[i] = file[i][(file[i]['pickup_longitude'] >= -74.242330) \
                                & (file[i]['pickup_longitude'] <= -73.717047) \
                                & (file[i]['pickup_latitude'] >= 40.560445) \
                                & (file[i]['pickup_latitude'] <= 40.908524) \
                                &(file[i]['dropoff_longitude'] >= -74.242330) \
                                & (file[i]['dropoff_longitude'] <= -73.717047) \
                                & (file[i]['dropoff_latitude'] >= 40.560445) \
                                & (file[i]['dropoff_latitude'] <= 40.908524)]
    file_res = pd.DataFrame()
    for i in range(len(file)):
        file_res = pd.concat([file_res,file[i].sample(round(year_num[n+6]/12+1))], axis = 0)
    file_res = file_res.sample(year_num[n+6])
    file_res['distance'] = file_res.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)
    
    return file_res


In [21]:
file2015_res = get_and_clean_data(0)

In [22]:
file2014_res = get_and_clean_data(1)

In [23]:
file2013_res = get_and_clean_data(2)

In [24]:
file2012_res = get_and_clean_data(3)

In [25]:
file2011_res = get_and_clean_data(4)

In [26]:
file2010_res = get_and_clean_data2(0)

In [27]:
file2009_res = get_and_clean_data3(0)

In [28]:
yellow_taxi = pd.DataFrame()
for i in [file2015_res, file2014_res, file2013_res,file2012_res,file2011_res,file2010_res,file2009_res]:
    yellow_taxi = pd.concat([yellow_taxi,i], axis = 0)

In [29]:
yellow_taxi = yellow_taxi.reset_index()
yellow_taxi = yellow_taxi.drop(['index'], axis =1 )

In [30]:
yellow_taxi.isnull().sum()

pickup_datetime      0
tip_amount           0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
distance             0
dtype: int64